In [4]:
import pandas as pd

# Dataset
We use the ECG dataset available [here](https://www.kaggle.com/shayanfazeli/heartbeat/data?select=mitbih_train.csv). There is [an article](https://arxiv.org/pdf/1805.00794.pdf) that uses the dataset, which we can use as a reference. More details about the dataset can be found [here](https://physionet.org/content/apnea-ecg/1.0.0/)

## Load the datasets

In [8]:
train = pd.read_csv('data/mitbih_train.csv', low_memory=False)
train.shape

(87553, 188)

In [9]:
test = pd.read_csv('data/mitbih_test.csv', low_memory=False)
test.shape

(21891, 188)

In [10]:
normal = pd.read_csv('data/ptbdb_normal.csv',low_memory=False)
normal.shape

(4045, 188)

In [12]:
abnormal = pd.read_csv('data/ptbdb_abnormal.csv', low_memory=False)
abnormal.shape

(10505, 188)